In [5]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import acebootlegsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from webUtils import getWebData, getHTML, getURL
from artistsRM import artistsRM
from artistRM import artistRM
from albums import albums
from time import sleep
import time
from multiprocessing import Pool
import re



import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-03-27 15:49:02.045459


# Initialize

In [6]:
%load_ext autoreload
%autoreload
discRM = discogs('rateyourmusic')
dutils = acebootlegsUtils()
artRM  = artistRM()
artsRM = artistsRM(discRM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download Known Artists

In [46]:
def getMyMusicDBMap():
    myMusicMap = getFile(ifile="/Users/tgadfort/Documents/code/music/myMusicMap.p")
    print("Found {0} entries".format(len(myMusicMap)))
    return myMusicMap

myMusicMap = getMyMusicDBMap()

Found 3942 entries


In [53]:
def downloadSearchResults(maxI=-1):
    for i,artistName in enumerate(myMusicMap.keys()):
        if i <= maxI:
            continue
        print("====>",i,"/",len(myMusicMap),"  : ",artistName)
        artsRM.searchRateYourMusicForArtist(artistName, sleeptime=10)
        break
        sleep(5)

In [54]:
downloadSearchResults(3)

====> 4 / 3942   :  Abdullah


===================== Searching For Abdullah =====================
	Search URL: https://rateyourmusic.com/search?searchtype=a&searchterm=Abdullah


HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
%load_ext autoreload
%autoreload
discRM = discogs('rateyourmusic')
dutils = rateyourmusicUtils()
artsRM = artistsRM(discRM)
artRM  = artistRM()

retval = artRM.getData("rym.p")

In [52]:
artsRM.searchRateYourMusicForArtist("Beatles")



===================== Searching For Beatles =====================
	Search URL: https://rateyourmusic.com/search?searchtype=a&searchterm=Beatles


HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
retval.artist.get()

In [ ]:
retval.url.get()

In [ ]:
retval.ID.get()

In [ ]:
retval.profile.get()

In [ ]:
for mediaType,mediaTypeData in retval.media.media.items():
    for album in mediaTypeData:
        print(mediaType,'\t',album.get())

In [ ]:
retval.mediaCounts.get()

In [2]:
from ioUtils import getFile, saveFile

In [36]:
html = "/Users/tgadfort/Downloads/Dave Matthews Band Albums: songs, discography, biography, and listening guide - Rate Your Music.html"
html = "/Users/tgadfort/Downloads/Øresund Space Collective Albums: songs, discography, biography, and listening guide - Rate Your Musi.html"
html = "/Users/tgadfort/Downloads/The Beatles Albums: songs, discography, biography, and listening guide - Rate Your Music.html"
html = "/Users/tgadfort/Downloads/Kenny G Albums: songs, discography, biography, and listening guide - Rate Your Music.html"
html = "/Users/tgadfort/Downloads/Peter Gabriel Albums: songs, discography, biography, and listening guide - Rate Your Music.html"
html = "/Users/tgadfort/Downloads/Fleetwood Mac Albums: songs, discography, biography, and listening guide - Rate Your Music.html"
html = "/Users/tgadfort/Downloads/Robbie Williams Albums: songs, discography, biography, and listening guide - Rate Your Music.html"
htmldata = getFile(html)

In [37]:
artRM = artistRM()
retval = artRM.getData(html)
artistID = retval.ID.ID
savename = artsRM.getArtistSavename(artistID)

In [38]:
saveFile(idata=htmldata, ifile=savename, debug=True)

Saving data to /Volumes/Biggy/Discog/artists-rateyourmusic/45/745.p
  --> This file is 41.6kB.
Saved data to /Volumes/Biggy/Discog/artists-rateyourmusic/45/745.p
  --> This file is 41.6kB.


'/Volumes/Biggy/Discog/artists-rateyourmusic/98/98.p'

In [ ]:
savename = "rym_search.p"
artsRM.downloadArtistURL(url=url, savename=savename, parse=False)

In [ ]:
bsdata = getHTML("rym_search.p")

In [ ]:
results = []

tables = bsdata.findAll("table") #<tr class="infobox">")
for i,table in enumerate(tables):
    isInfoBox = table.find("tr", {"class": "infobox"})
    if isInfoBox is None:
        continue
    if i % 2 == 0:
        continue
    for j,subtable in enumerate(table.findAll("table")):
        for tr in subtable.findAll("tr"):
            values   = artRM.getNamesAndURLs(tr)
            name     = values[0].name
            url      = values[0].url
            ref      = tr.find("a")
            artistID = dutils.getArtistID(ref.attrs['title'])
            results.append({"Name": name, "URL": url, "ID": artistID})
            
results

# Artists From Search

In [ ]:
dbFull  = ["Discogs", "AllMusic"]
dbNames = [x.lower() for x in dbFull]
dbKeys  = []
for item in list(zip(dbFull, dbNames)):
    dbKeys.append({"Key": item[0], "Name": item[1]})
print(dbKeys)


def getMyMusicDBMap():
    myMusicMap = getFile(ifile="/Users/tgadfort/Documents/code/music/myMusicMap.p")
    print("Found {0} entries".format(len(myMusicMap)))
    showMyMusicMap(myMusicMap)
    return myMusicMap


def showMyMusicMap(myMusicMap):
    cntrs = {"Discogs": 0, "AllMusic": 0, "MusicBrainz": 0}
    for myArtistName, myArtistData in myMusicMap.items():
        if myArtistData["Discogs"] is not None:
            cntrs["Discogs"] += 1
        if myArtistData["AllMusic"] is not None:
            cntrs["AllMusic"] += 1
        if myArtistData["MusicBrainz"] is not None:
            cntrs["MusicBrainz"] += 1
    print(cntrs)

In [ ]:
myMusicMap = getMyMusicDBMap()

In [ ]:
%load_ext autoreload
%autoreload
discMB = discogs('musicbrainz')
artsMB = artistsMB(discMB)

for i,(artistName,artistData) in enumerate(myMusicMap.items()):
    if i <= 683:
        continue
    try:
        mbval = artistData.get("MusicBrainz")
    except:
        mbval = None

    if mbval is not None:
        continue
    print("===========",i,'/',len(myMusicMap),'\t',artistName,"===========")
    artsMB.searchMusicBrainzForArtist(artistName)
    sleep(3)

In [ ]:
from webUtils import getWebData
url="https://musicbrainz.org/artist/cc197bad-dc9c-440d-a5b5-d52ba2e14234"
savename="musicbrainz.p"
getWebData(base=url, savename=savename, useSafari=False)

# Testing

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

In [ ]:
artsAM.searchAllMusicForArtist('Creed')

In [ ]:
artistRef = "https://www.allmusic.com/artist/creed-warner-mn0003293932"
artistRef = "{0}/discography".format(artistRef)
artistRef
#urllib.parse.urljoin(artistRef) #, "/discography")

In [ ]:
#from discogsUtils import allmusicUtils
# Basic stuff
%load_ext autoreload
%autoreload
from discogsUtils import allmusicUtils

discUtils = allmusicUtils()
artistID = discUtils.getArtistID('https://www.allmusic.com/artist/bliss-mn0001514655', debug=True)
print("retval = ",artistID)

In [ ]:
allMusicIDMap = {}
savename = "collections/allMusicIDMap.p"
allMusicIDMap = getFile(savename)
len(allMusicIDMap)

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')
artsAM = artistsAM(discAM)

In [ ]:
allMusicIDMap['MN0000986327']

In [ ]:
from time import sleep
for i,(artistID,artistVals) in enumerate(allMusicIDMap.items()):
    if i <= 49496:
        continue
    artistName,ref = artistVals
    print(i,'/',len(allMusicIDMap),'\t',artistName,'\t',ref)
    url      = artsAM.getArtistURL(ref)
    savename = artsAM.getArtistSavename(artistID)
    artsAM.downloadArtistURL(url, savename, force=True)

In [ ]:
data = artsAM.getData('/Users/tgadfort/Music/Discog/artists-allmusic/62/0000567862.p')

In [ ]:
file = "/Volumes/Music/Discog/artists-allmusic/68/0000782768.p"
file = "MN0000961234.p"
file = "/Users/tgadfort/Music/Discog/artists-allmusic/27/0000986327.p"
artAM = artistAM()
data = artAM.getData(file)

In [ ]:
data

In [ ]:
print("Artist")
info = data.artist.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("URL")
info = data.url.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("ID")
info = data.ID.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Pages")
info = data.pages.get()
for k,v in info.items():
    print("\t",k,'\t',v)

print("Profile")
info = data.profile.get()
for k,v in info.items():
    if k == "profile":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)
    elif k == "members":
        print("\t",k)
        for v2 in v:
            print("\t\t",v2)
    else:
        print('\t',k,'\t',v)
            

print("Media")
info = data.media.get()
for k,v in info.items():
    if k == "media":
        print("\t",k)
        for k2,v2 in v.items():
            print("\t\t",k2)
            for v3 in v2:
                for k4,v4 in v3.get().items():
                    if v4 is None:
                        print("\t\t\t",k4,'\t',v4)
                    elif isinstance(v4, str):
                        print("\t\t\t",k4,'\t',v4)
                    else:
                        print("\t\t\t",k4)
                        for k5,v5 in v4.get().items():
                            print("\t\t\t\t",k5,'\t',v5)


print("MediaCounts")
info = data.mediaCounts.get()
for k,v in info.items():
    print("\t",k,'\t',v)